"""Converting decades_tcc to the standard."""

In [1]:
import pandas as pd
import numpy as np
import s3fs
import os
import io
import boto3

import s3fs
fs = s3fs.S3FileSystem(anon=False,key='###',secret='###')

from dotenv import load_dotenv
load_dotenv(verbose=True)

def aws_session(region_name='us-east-1'):
    return boto3.session.Session(aws_access_key_id=os.getenv('AWS_ACCESS_KEY_ID'), #looks for any .env file
                                aws_secret_access_key=os.getenv('AWS_ACCESS_KEY_SECRET'), #Has to be in same directory
                                region_name=region_name) #from above

def make_bucket(name, acl): 
    session = aws_session()
    s3_resource = session.resource('s3')
    return s3_resource.create_bucket(Bucket=name, ACL=acl)

def upload_file_to_bucket(bucket_name, file_path):
    session = aws_session()
    s3_resource = session.resource('s3')
    file_dir, file_name = os.path.split(file_path)

    bucket = s3_resource.Bucket(bucket_name)
    bucket.upload_file(
      Filename=file_path,
      Key=file_name,
      ExtraArgs={'ACL': 'public-read'}
    )

    s3_url = f"https://{bucket_name}.s3.amazonaws.com/{file_name}"
    return s3_url

def download_file_from_bucket(bucket_name, s3_key, dst_path):
    session = aws_session()
    s3_resource = session.resource('s3')
    bucket = s3_resource.Bucket(bucket_name)
    bucket.download_file(Key=s3_key, Filename=dst_path)

## download_file_from_bucket('music-demo-lyrics', 'lyrics_25k.csv', 'short_name.csv')
## with open('short_name.csv') as fo:
    ## print(fo.read())

In [4]:
!aws s3 ls

2021-04-06 21:03:54 music-demo-lyrics
2021-04-06 20:10:43 music-lyrics
2021-04-07 07:23:53 worm-begin


In [6]:
!aws s3 ls 'worm-begin'

2021-04-07 08:38:33     105621 album_details_25k.csv
2021-04-07 08:09:22   27655251 decades_tcc_ceds_music.csv
2021-04-07 08:34:22     348120 genres_artists_data.csv
2021-04-07 08:19:38  276163416 genres_lyrics_data.csv
2021-04-07 08:22:31  187407184 labeled_lyrics_cleaned.csv
2021-04-07 08:38:18   40754845 lyrics_25k.csv
2021-04-07 08:38:34    2276377 songs_details_25k.csv


In [7]:
decades_df = pd.read_csv('s3://worm-begin/decades_tcc_ceds_music.csv')

In [8]:
decades_df

,Unnamed: 0,artist_name,track_name,release_date,genre,lyrics,len,dating,violence,world/life,...,sadness,feelings,danceability,loudness,acousticness,instrumentalness,valence,energy,topic,age
0,0,mukesh,mohabbat bhi jhoothi,1950,pop,hold time feel break feel untrue convince spea...,95,0.000598,0.063746,0.000598,...,0.380299,0.117175,0.357739,0.454119,0.997992,0.901822,0.339448,0.137110,sadness,1.000000
1,4,frankie laine,i believe,1950,pop,believe drop rain fall grow believe darkest ni...,51,0.035537,0.096777,0.443435,...,0.001284,0.001284,0.331745,0.647540,0.954819,0.000002,0.325021,0.263240,world/life,1.000000
2,6,johnnie ray,cry,1950,pop,sweetheart send letter goodbye secret feel bet...,24,0.002770,0.002770,0.002770,...,0.002770,0.225422,0.456298,0.585288,0.840361,0.000000,0.351814,0.139112,music,1.000000
3,10,pérez prado,patricia,1950,pop,kiss lips want stroll charm mambo chacha merin...,54,0.048249,0.001548,0.001548,...,0.225889,0.001548,0.686992,0.744404,0.083935,0.199393,0.775350,0.743736,romantic,1.000000
4,12,giorgos papadopoulos,apopse eida oneiro,1950,pop,till darling till matter know till dream live ...,48,0.001350,0.001350,0.417772,...,0.068800,0.001350,0.291671,0.646489,0.975904,0.000246,0.597073,0.394375,romantic,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28367,82447,mack 10,10 million ways,2019,hip hop,cause fuck leave scar tick tock clock come kno...,78,0.001350,0.001350,0.001350,...,0.065664,0.001350,0.889527,0.759711,0.062549,0.000000,0.751649,0.695686,obscene,0.014286
28368,82448,m.o.p.,ante up (robbin hoodz theory),2019,hip hop,minks things chain ring braclets yap fame come...,67,0.001284,0.001284,0.035338,...,0.001284,0.001284,0.662082,0.789580,0.004607,0.000002,0.922712,0.797791,obscene,0.014286
28369,82449,nine,whutcha want?,2019,hip hop,get ban get ban stick crack relax plan attack ...,77,0.001504,0.154302,0.168988,...,0.001504,0.001504,0.663165,0.726970,0.104417,0.000001,0.838211,0.767761,obscene,0.014286
28370,82450,will smith,switch,2019,hip hop,check check yeah yeah hear thing call switch g...,67,0.001196,0.001196,0.001196,...,0.001196,0.001196,0.883028,0.786888,0.007027,0.000503,0.508450,0.885882,obscene,0.014286


In [9]:
decades_df.describe(include='all')

,Unnamed: 0,artist_name,track_name,release_date,genre,lyrics,len,dating,violence,world/life,...,sadness,feelings,danceability,loudness,acousticness,instrumentalness,valence,energy,topic,age
count,28372.000000,28372,28372,28372.000000,28372,28372,28372.000000,28372.000000,28372.000000,28372.000000,...,28372.000000,28372.000000,28372.000000,28372.000000,2.837200e+04,28372.000000,28372.000000,28372.000000,28372,28372.000000
unique,NaN,5426,23689,NaN,7,28372,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8,NaN
top,NaN,johnny cash,tonight,NaN,pop,shall tell start friends yippee mesa dupree do...,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,sadness,NaN
freq,NaN,190,17,NaN,7042,1,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6096,NaN
mean,42946.323558,NaN,NaN,1990.236888,NaN,NaN,73.028444,0.021112,0.118396,0.120973,...,0.129389,0.030996,0.533348,0.665249,3.392347e-01,0.080049,0.532864,0.569875,NaN,0.425187
std,24749.325492,NaN,NaN,18.487463,NaN,NaN,41.829831,0.052370,0.178684,0.172200,...,0.181143,0.071652,0.173218,0.108434,3.267143e-01,0.211245,0.250972,0.244385,NaN,0.264107
min,0.000000,NaN,NaN,1950.000000,NaN,NaN,1.000000,0.000291,0.000284,0.000291,...,0.000284,0.000289,0.005415,0.000000,2.811248e-07,0.000000,0.000000,0.000000,NaN,0.014286
25%,20391.250000,NaN,NaN,1975.000000,NaN,NaN,42.000000,0.000923,0.001120,0.001170,...,0.001144,0.000993,0.412975,0.595364,3.423598e-02,0.000000,0.329143,0.380361,NaN,0.185714
50%,45405.500000,NaN,NaN,1991.000000,NaN,NaN,63.000000,0.001462,0.002506,0.006579,...,0.005263,0.001754,0.538612,0.679050,2.259028e-01,0.000085,0.539365,0.580567,NaN,0.414286
75%,64090.500000,NaN,NaN,2007.000000,NaN,NaN,93.000000,0.004049,0.192608,0.197793,...,0.235113,0.032622,0.656666,0.749026,6.325298e-01,0.009335,0.738252,0.772766,NaN,0.642857


In [10]:
decades_df.keys()

Index(['Unnamed: 0', 'artist_name', 'track_name', 'release_date', 'genre',
       'lyrics', 'len', 'dating', 'violence', 'world/life', 'night/time',
       'shake the audience', 'family/gospel', 'romantic', 'communication',
       'obscene', 'music', 'movement/places', 'light/visual perceptions',
       'family/spiritual', 'like/girls', 'sadness', 'feelings', 'danceability',
       'loudness', 'acousticness', 'instrumentalness', 'valence', 'energy',
       'topic', 'age'],
      dtype='object')

In [17]:
decades2_df = pd.DataFrame((decades_df), columns=['original_csv','artist_name','track_name','link','lyrics','language','genre','release_date'])

In [18]:
decades2_df

,original_csv,artist_name,track_name,link,lyrics,language,genre,release_date
0,NaN,mukesh,mohabbat bhi jhoothi,NaN,hold time feel break feel untrue convince spea...,NaN,pop,1950
1,NaN,frankie laine,i believe,NaN,believe drop rain fall grow believe darkest ni...,NaN,pop,1950
2,NaN,johnnie ray,cry,NaN,sweetheart send letter goodbye secret feel bet...,NaN,pop,1950
3,NaN,pérez prado,patricia,NaN,kiss lips want stroll charm mambo chacha merin...,NaN,pop,1950
4,NaN,giorgos papadopoulos,apopse eida oneiro,NaN,till darling till matter know till dream live ...,NaN,pop,1950
...,...,...,...,...,...,...,...,...
28367,NaN,mack 10,10 million ways,NaN,cause fuck leave scar tick tock clock come kno...,NaN,hip hop,2019
28368,NaN,m.o.p.,ante up (robbin hoodz theory),NaN,minks things chain ring braclets yap fame come...,NaN,hip hop,2019
28369,NaN,nine,whutcha want?,NaN,get ban get ban stick crack relax plan attack ...,NaN,hip hop,2019
28370,NaN,will smith,switch,NaN,check check yeah yeah hear thing call switch g...,NaN,hip hop,2019


In [19]:
decades2_df['original_csv'] = 'decades_tcc'

In [22]:
decades3_df = decades2_df.rename(columns={'track_name':'song_name','release_date':'date'})

In [23]:
decades3_df

,original_csv,artist_name,song_name,link,lyrics,language,genre,date
0,decades_tcc,mukesh,mohabbat bhi jhoothi,NaN,hold time feel break feel untrue convince spea...,NaN,pop,1950
1,decades_tcc,frankie laine,i believe,NaN,believe drop rain fall grow believe darkest ni...,NaN,pop,1950
2,decades_tcc,johnnie ray,cry,NaN,sweetheart send letter goodbye secret feel bet...,NaN,pop,1950
3,decades_tcc,pérez prado,patricia,NaN,kiss lips want stroll charm mambo chacha merin...,NaN,pop,1950
4,decades_tcc,giorgos papadopoulos,apopse eida oneiro,NaN,till darling till matter know till dream live ...,NaN,pop,1950
...,...,...,...,...,...,...,...,...
28367,decades_tcc,mack 10,10 million ways,NaN,cause fuck leave scar tick tock clock come kno...,NaN,hip hop,2019
28368,decades_tcc,m.o.p.,ante up (robbin hoodz theory),NaN,minks things chain ring braclets yap fame come...,NaN,hip hop,2019
28369,decades_tcc,nine,whutcha want?,NaN,get ban get ban stick crack relax plan attack ...,NaN,hip hop,2019
28370,decades_tcc,will smith,switch,NaN,check check yeah yeah hear thing call switch g...,NaN,hip hop,2019


In [24]:
decades3_df[decades3_df.duplicated()]

,original_csv,artist_name,song_name,link,lyrics,language,genre,date


""" I see the date here in decades, and I'd like to add that back into the genres main csv.  The genres series has a csv with date (by artist, not song, unfortunately). But I need to strip the "/" s from the singer_name.  And take off 'Lyrics' from the end of the artist name in the genres csv with years in it."""

"""WARNING!: Dead end just ahead...but I did learn some regex."""

In [26]:
merged2_df = pd.read_csv('merged2_genre_df.csv')

In [38]:
merged2_df

,original_csv,artist_name,song_name,link,lyrics,language,genre
0,genres_csv,/10000-maniacs/,More Than This,/10000-maniacs/more-than-this.html,I could feel at the time. There was no way of ...,ENGLISH,Rock
1,genres_csv,/10000-maniacs/,More Than This,/10000-maniacs/more-than-this.html,I could feel at the time. There was no way of ...,ENGLISH,Pop
2,genres_csv,/10000-maniacs/,Because The Night,/10000-maniacs/because-the-night.html,"Take me now, baby, here as I am. Hold me close...",ENGLISH,Rock
3,genres_csv,/10000-maniacs/,Because The Night,/10000-maniacs/because-the-night.html,"Take me now, baby, here as I am. Hold me close...",ENGLISH,Pop
4,genres_csv,/10000-maniacs/,These Are Days,/10000-maniacs/these-are-days.html,These are. These are days you'll remember. Nev...,ENGLISH,Rock
...,...,...,...,...,...,...,...
124225,genres_csv,/sambo/,Valerie,/sambo/valerie.html,Well sometimes I go out by myself. And I look ...,ENGLISH,Rock
124226,genres_csv,/sambo/,Wake Me Up,/sambo/wake-me-up.html,Feeling my way through the darkness. Guided by...,ENGLISH,Samba
124227,genres_csv,/sambo/,Wake Me Up,/sambo/wake-me-up.html,Feeling my way through the darkness. Guided by...,ENGLISH,Rock
124228,genres_csv,/seu-jorge/,Don't,/seu-jorge/dont.html,"Don't, don't, that's what you say. Each time t...",ENGLISH,Samba


In [34]:
punct = "\n\r"+string.punctuation
merged2_df['artist_name'] = merged2_df['artist_name'].str.translate(str.maketrans('','',punct))

AttributeError: type object 'str' has no attribute 'punctuation'

In [39]:
merged3_df = merged2_df['artist_name'].str.replace(r'[^\w]', ' ')

<ipython-input-39-9eb4c0d0d6d4>:1: FutureWarning: The default value of regex will change from True to False in a future version.
  merged3_df = merged2_df['artist_name'].str.replace(r'[^\w]', ' ')


In [37]:
merged3_df

0          10000maniacs
1          10000maniacs
2          10000maniacs
3          10000maniacs
4          10000maniacs
              ...      
124225            sambo
124226            sambo
124227            sambo
124228         seujorge
124229    zecapagodinho
Name: artist_name, Length: 124230, dtype: object

In [28]:
download_file_from_bucket('worm-begin', 'album_details_25k.csv', 'year_df.csv')

In [30]:
year_df = pd.read_csv('year_df.csv')

In [40]:
year_df

,Unnamed: 0,id,singer_name,name,type,year
0,0,5765.0,Taylor Swift Lyrics,Taylor Swift,album,2006
1,1,6432.0,Taylor Swift Lyrics,Sounds Of The Season: The Taylor Swift Holiday...,EP,2007
2,2,6995.0,Taylor Swift Lyrics,Fearless,album,2008
3,3,10358.0,Taylor Swift Lyrics,Speak Now,album,2010
4,4,24353.0,Taylor Swift Lyrics,Red,album,2012
...,...,...,...,...,...,...
1814,1814,22624.0,Jason Mraz Lyrics,Love Is A Four Letter Word,album,2012
1815,1815,32642.0,Jason Mraz Lyrics,Yes!,album,2014
1816,1816,58384.0,Jason Mraz Lyrics,Know.,album,2018
1817,1817,81696.0,Jason Mraz Lyrics,Look For The Good,album,2020


In [32]:
pat = r'\b(?:{})\b'.format('|'.join(remove_words))

pat
'\\b(?:Lyrics)\\b'

year_df['singer_name'] = year_df['singer_name'].str.replace(pat, '')

NameError: name 'remove_words' is not defined

"""Trying some regex work now""" 

In [43]:
import numpy as np
year_df['test'] = year_df['singer_name'].str.extract(r'(^w{7})')
year_df

,Unnamed: 0,id,singer_name,name,type,year,test
0,0,5765.0,Taylor Swift Lyrics,Taylor Swift,album,2006,NaN
1,1,6432.0,Taylor Swift Lyrics,Sounds Of The Season: The Taylor Swift Holiday...,EP,2007,NaN
2,2,6995.0,Taylor Swift Lyrics,Fearless,album,2008,NaN
3,3,10358.0,Taylor Swift Lyrics,Speak Now,album,2010,NaN
4,4,24353.0,Taylor Swift Lyrics,Red,album,2012,NaN
...,...,...,...,...,...,...,...
1814,1814,22624.0,Jason Mraz Lyrics,Love Is A Four Letter Word,album,2012,NaN
1815,1815,32642.0,Jason Mraz Lyrics,Yes!,album,2014,NaN
1816,1816,58384.0,Jason Mraz Lyrics,Know.,album,2018,NaN
1817,1817,81696.0,Jason Mraz Lyrics,Look For The Good,album,2020,NaN


"""str.extract was a bust.  Is it not a str?"""

In [44]:
merged2_df.replace('(/)','',regex=True, inplace = True)
merged2_df

,original_csv,artist_name,song_name,link,lyrics,language,genre
0,genres_csv,10000-maniacs,More Than This,10000-maniacsmore-than-this.html,I could feel at the time. There was no way of ...,ENGLISH,Rock
1,genres_csv,10000-maniacs,More Than This,10000-maniacsmore-than-this.html,I could feel at the time. There was no way of ...,ENGLISH,Pop
2,genres_csv,10000-maniacs,Because The Night,10000-maniacsbecause-the-night.html,"Take me now, baby, here as I am. Hold me close...",ENGLISH,Rock
3,genres_csv,10000-maniacs,Because The Night,10000-maniacsbecause-the-night.html,"Take me now, baby, here as I am. Hold me close...",ENGLISH,Pop
4,genres_csv,10000-maniacs,These Are Days,10000-maniacsthese-are-days.html,These are. These are days you'll remember. Nev...,ENGLISH,Rock
...,...,...,...,...,...,...,...
124225,genres_csv,sambo,Valerie,sambovalerie.html,Well sometimes I go out by myself. And I look ...,ENGLISH,Rock
124226,genres_csv,sambo,Wake Me Up,sambowake-me-up.html,Feeling my way through the darkness. Guided by...,ENGLISH,Samba
124227,genres_csv,sambo,Wake Me Up,sambowake-me-up.html,Feeling my way through the darkness. Guided by...,ENGLISH,Rock
124228,genres_csv,seu-jorge,Don't,seu-jorgedont.html,"Don't, don't, that's what you say. Each time t...",ENGLISH,Samba


"""One down.  One to go."""
"""Tried a couple different .iloc and .loc combinations.  Trying to avoid making changes to a slice warnings.  Didn't work."""

In [47]:
year_df.loc['singer_name'].replace(('Lyrics'), '', regex = True, inplace = True)

KeyError: 'singer_name'

In [49]:
year_df['singer_name'].replace(('Lyrics'), '', regex = True, inplace = True)

In [50]:
year_df

,Unnamed: 0,id,singer_name,name,type,year,test
0,0,5765.0,Taylor Swift,Taylor Swift,album,2006,NaN
1,1,6432.0,Taylor Swift,Sounds Of The Season: The Taylor Swift Holiday...,EP,2007,NaN
2,2,6995.0,Taylor Swift,Fearless,album,2008,NaN
3,3,10358.0,Taylor Swift,Speak Now,album,2010,NaN
4,4,24353.0,Taylor Swift,Red,album,2012,NaN
...,...,...,...,...,...,...,...
1814,1814,22624.0,Jason Mraz,Love Is A Four Letter Word,album,2012,NaN
1815,1815,32642.0,Jason Mraz,Yes!,album,2014,NaN
1816,1816,58384.0,Jason Mraz,Know.,album,2018,NaN
1817,1817,81696.0,Jason Mraz,Look For The Good,album,2020,NaN


'''Yes!!"""

In [52]:
year2_df = pd.DataFrame((year_df), columns=['singer_name','year'])
year2_df

,singer_name,year
0,Taylor Swift,2006
1,Taylor Swift,2007
2,Taylor Swift,2008
3,Taylor Swift,2010
4,Taylor Swift,2012
...,...,...
1814,Jason Mraz,2012
1815,Jason Mraz,2014
1816,Jason Mraz,2018
1817,Jason Mraz,2020


In [54]:
year2_df.groupby(['singer_name']).count()

,year
singer_name,
'N Sync,6
5 Seconds Of Summer (5SOS),11
50 Cent,20
6ix9ine (Tekashi69),3
A Boogie Wit Da Hoodie,8
...,...
Twenty One Pilots,6
Westlife,13
Wham!,3


In [55]:
merged2_df.groupby(['artist_name']).count()

,original_csv,song_name,link,lyrics,language,genre
artist_name,,,,,,
10000-maniacs,264,264,264,264,264,264
12-stones,100,100,100,100,100,100
2-chainz,102,102,102,102,102,102
21-savage,11,11,11,11,11,11
2ne1,48,48,48,48,48,48
...,...,...,...,...,...,...
zeca-pagodinho,1,1,1,1,1,1
zelia-duncan,3,3,3,3,3,3
zendaya,66,66,66,66,66,66


In [56]:
year2_df=year2_df.rename(columns={'singer_name':'artist_name'})
year2_df

,artist_name,year
0,Taylor Swift,2006
1,Taylor Swift,2007
2,Taylor Swift,2008
3,Taylor Swift,2010
4,Taylor Swift,2012
...,...,...
1814,Jason Mraz,2012
1815,Jason Mraz,2014
1816,Jason Mraz,2018
1817,Jason Mraz,2020


"""so, there are 148 artists in year_df, and ~10 times that amount in merged_df.  At this point, I'm doing it for the training."""

In [57]:
merged2_df

,original_csv,artist_name,song_name,link,lyrics,language,genre
0,genres_csv,10000-maniacs,More Than This,10000-maniacsmore-than-this.html,I could feel at the time. There was no way of ...,ENGLISH,Rock
1,genres_csv,10000-maniacs,More Than This,10000-maniacsmore-than-this.html,I could feel at the time. There was no way of ...,ENGLISH,Pop
2,genres_csv,10000-maniacs,Because The Night,10000-maniacsbecause-the-night.html,"Take me now, baby, here as I am. Hold me close...",ENGLISH,Rock
3,genres_csv,10000-maniacs,Because The Night,10000-maniacsbecause-the-night.html,"Take me now, baby, here as I am. Hold me close...",ENGLISH,Pop
4,genres_csv,10000-maniacs,These Are Days,10000-maniacsthese-are-days.html,These are. These are days you'll remember. Nev...,ENGLISH,Rock
...,...,...,...,...,...,...,...
124225,genres_csv,sambo,Valerie,sambovalerie.html,Well sometimes I go out by myself. And I look ...,ENGLISH,Rock
124226,genres_csv,sambo,Wake Me Up,sambowake-me-up.html,Feeling my way through the darkness. Guided by...,ENGLISH,Samba
124227,genres_csv,sambo,Wake Me Up,sambowake-me-up.html,Feeling my way through the darkness. Guided by...,ENGLISH,Rock
124228,genres_csv,seu-jorge,Don't,seu-jorgedont.html,"Don't, don't, that's what you say. Each time t...",ENGLISH,Samba


In [59]:
new_main_genres_df = pd.merge(merged2_df, year2_df, on='artist_name')
new_main_genres_df

,original_csv,artist_name,song_name,link,lyrics,language,genre,year


In [60]:
year2_df[year2_df['artist_name'].str.match('Jason Mraz')== True]

,artist_name,year
1808,Jason Mraz,2002
1809,Jason Mraz,2002
1810,Jason Mraz,2004
1811,Jason Mraz,2005
1812,Jason Mraz,2008
1813,Jason Mraz,2010
1814,Jason Mraz,2012
1815,Jason Mraz,2014
1816,Jason Mraz,2018
1817,Jason Mraz,2020


In [66]:
merged2_df[merged2_df['artist_name'].str.match('Johnny Cash')== True]

,original_csv,artist_name,song_name,link,lyrics,language,genre


"""I have been trying to pd.merge two csv files from different datasets.  There wasn't a Union between 'artist_name' s"""

In [67]:
decades3_df

,original_csv,artist_name,song_name,link,lyrics,language,genre,date
0,decades_tcc,mukesh,mohabbat bhi jhoothi,NaN,hold time feel break feel untrue convince spea...,NaN,pop,1950
1,decades_tcc,frankie laine,i believe,NaN,believe drop rain fall grow believe darkest ni...,NaN,pop,1950
2,decades_tcc,johnnie ray,cry,NaN,sweetheart send letter goodbye secret feel bet...,NaN,pop,1950
3,decades_tcc,pérez prado,patricia,NaN,kiss lips want stroll charm mambo chacha merin...,NaN,pop,1950
4,decades_tcc,giorgos papadopoulos,apopse eida oneiro,NaN,till darling till matter know till dream live ...,NaN,pop,1950
...,...,...,...,...,...,...,...,...
28367,decades_tcc,mack 10,10 million ways,NaN,cause fuck leave scar tick tock clock come kno...,NaN,hip hop,2019
28368,decades_tcc,m.o.p.,ante up (robbin hoodz theory),NaN,minks things chain ring braclets yap fame come...,NaN,hip hop,2019
28369,decades_tcc,nine,whutcha want?,NaN,get ban get ban stick crack relax plan attack ...,NaN,hip hop,2019
28370,decades_tcc,will smith,switch,NaN,check check yeah yeah hear thing call switch g...,NaN,hip hop,2019


In [68]:
decades3_df.to_csv('decades3_df.csv', index= False)

In [69]:
upload_file_to_bucket('music-demo-lyrics','decades3_df.csv')

'https://music-demo-lyrics.s3.amazonaws.com/decades3_df.csv'